In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pds_xls = pd.read_excel("6Li_with_Daejeon16_gs_energy_from_Dropbox.xlsx")

In [ ]:
pds_xls.head()

In [ ]:
pds_xls.shape

In [ ]:
pds_xls.pop(pds_xls.columns[1])
pds_xls.pop(pds_xls.columns[len(pds_xls.columns)-1])

In [ ]:
pds_xls

In [ ]:
# ds_xls.pop(pds_xls.columns[0])


In [ ]:
data  = pds_xls.to_numpy()
print(data.shape)

In [ ]:
datum = {'Nmax': N_Max, 'data': data}

In [ ]:
np.save( 'processed_extrapolation.npy', datum)

In [256]:
import numpy as np
X = np.load('processed_extrapolation.npy', allow_pickle=True)


In [257]:
print(X[()]['data'])
print(X[()]['Nmax'])


[[  8.      -16.57529 -22.07245 -25.76041 -28.26697 -29.85611 -30.82073
  -31.37646 -31.68395 -31.8479 ]
 [  9.      -18.76554 -24.21102 -27.54317 -29.59771 -30.75878 -31.38568
  -31.70678 -31.8667  -31.94453]
 [ 10.      -20.62863 -25.90418 -28.81887 -30.44101 -31.25914 -31.65792
  -31.84509 -31.93299 -31.97466]
 [ 12.5     -23.92152 -28.51269 -30.44546 -31.30909 -31.67491 -31.8374
  -31.91427 -31.95437 -31.97709]
 [ 15.      -25.57763 -29.50027 -30.86382 -31.43441 -31.68819 -31.81424
  -31.88416 -31.92669 -31.95399]
 [ 17.5     -26.05861 -29.57761 -30.78639 -31.32032 -31.58441 -31.73186
  -31.82064 -31.87769 -31.91603]
 [ 20.      -25.7044  -29.14255 -30.45505 -31.07796 -31.40886 -31.60391
  -31.72595 -31.8062  -31.86106]
 [ 22.5     -24.67286 -28.36076 -29.931   -30.72161 -31.16096 -31.42602
  -31.59523 -31.70796 -31.78564]
 [ 25.      -23.01104 -27.27792 -29.22278 -30.2445  -30.8306  -31.18986
  -31.42179 -31.57754 -31.68565]
 [ 27.5     -20.7303  -25.88942 -28.32302 -29.63613 -30.

In [258]:
# We have to figure out a network now, the idea is to get an ODE to integrate this.

In [259]:
import math
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.color_palette("bright")
import matplotlib as mpl
import matplotlib.cm as cm

import torch
from torch import Tensor
from torch import nn
from torch.nn  import functional as F 
from torch.autograd import Variable

use_cuda = torch.cuda.is_available()


def ode_solve(z0, t0, t1, f):
    """
    Simplest Euler ODE initial value solver
    """
    h_max = 0.05
    n_steps = math.ceil((abs(t1 - t0)/h_max).max().item())

    h = (t1 - t0)/n_steps
    t = t0
    z = z0

    for i_step in range(n_steps):
        z = z + h * f(z, t)
        t = t + h
    return z



## Let us now figure out how to get a model.
class ODEF(nn.Module):
    def forward_with_grad(self, z, t, grad_outputs):
        """Compute f and a df/dz, a df/dp, a df/dt"""
        batch_size = z.shape[0]

        out = self.forward(z, t)

        a = grad_outputs
        adfdz, adfdt, *adfdp = torch.autograd.grad(
            (out,), (z, t) + tuple(self.parameters()), grad_outputs=(a),
            allow_unused=True, retain_graph=True
        )
        # grad method automatically sums gradients for batch items, we have to expand them back
        if adfdp is not None:
            adfdp = torch.cat([p_grad.flatten()
                              for p_grad in adfdp]).unsqueeze(0)
            adfdp = adfdp.expand(batch_size, -1) / batch_size
        if adfdt is not None:
            adfdt = adfdt.expand(batch_size, 1) / batch_size
        return out, adfdz, adfdt, adfdp

    def flatten_parameters(self):
        p_shapes = []
        flat_parameters = []
        for p in self.parameters():
            p_shapes.append(p.size())
            flat_parameters.append(p.flatten())
        return torch.cat(flat_parameters)

In [260]:
class ODEAdjoint(torch.autograd.Function):
    @staticmethod
    def forward(ctx, z0, t, flat_parameters, func):
        assert isinstance(func, ODEF)
        bs, *z_shape = z0.size()
        time_len = t.size(0)

        with torch.no_grad():
            z = torch.zeros(time_len, bs, *z_shape).to(z0)
            z[0] = z0
            for i_t in range(time_len - 1):
                z0 = ode_solve(z0, t[i_t], t[i_t+1], func)
                z[i_t+1] = z0

        ctx.func = func
        ctx.save_for_backward(t, z.clone(), flat_parameters)
        return z

    @staticmethod
    def backward(ctx, dLdz):
        """
        dLdz shape: time_len, batch_size, *z_shape
        """
        func = ctx.func
        t, z, flat_parameters = ctx.saved_tensors
        time_len, bs, *z_shape = z.size()
        n_dim = np.prod(z_shape)
        n_params = flat_parameters.size(0)

        # Dynamics of augmented system to be calculated backwards in time
        def augmented_dynamics(aug_z_i, t_i):
            """
            tensors here are temporal slices
            t_i - is tensor with size: bs, 1
            aug_z_i - is tensor with size: bs, n_dim*2 + n_params + 1
            """
            z_i, a = aug_z_i[:, :n_dim], aug_z_i[:, n_dim:2*n_dim]  # ignore parameters and time

            # Unflatten z and a
            z_i = z_i.view(bs, *z_shape)
            a = a.view(bs, *z_shape)
            with torch.set_grad_enabled(True):
                t_i = t_i.detach().requires_grad_(True)
                z_i = z_i.detach().requires_grad_(True)
                func_eval, adfdz, adfdt, adfdp = func.forward_with_grad(z_i, t_i, grad_outputs=a)  # bs, *z_shape
                adfdz = adfdz.to(z_i) if adfdz is not None else torch.zeros(bs, *z_shape).to(z_i)
                adfdp = adfdp.to(z_i) if adfdp is not None else torch.zeros(bs, n_params).to(z_i)
                adfdt = adfdt.to(z_i) if adfdt is not None else torch.zeros(bs, 1).to(z_i)

            # Flatten f and adfdz
            func_eval = func_eval.view(bs, n_dim)
            adfdz = adfdz.view(bs, n_dim) 
            return torch.cat((func_eval, -adfdz, -adfdp, -adfdt), dim=1)

        dLdz = dLdz.view(time_len, bs, n_dim)  # flatten dLdz for convenience
        with torch.no_grad():
            ## Create placeholders for output gradients
            # Prev computed backwards adjoints to be adjusted by direct gradients
            adj_z = torch.zeros(bs, n_dim).to(dLdz)
            adj_p = torch.zeros(bs, n_params).to(dLdz)
            # In contrast to z and p we need to return gradients for all times
            adj_t = torch.zeros(time_len, bs, 1).to(dLdz)

            for i_t in range(time_len-1, 0, -1):
                z_i = z[i_t]
                t_i = t[i_t]
                f_i = func(z_i, t_i).view(bs, n_dim)

                # Compute direct gradients
                dLdz_i = dLdz[i_t]
                dLdt_i = torch.bmm(torch.transpose(dLdz_i.unsqueeze(-1), 1, 2), f_i.unsqueeze(-1))[:, 0]

                # Adjusting adjoints with direct gradients
                adj_z += dLdz_i
                adj_t[i_t] = adj_t[i_t] - dLdt_i

                # Pack augmented variable
                aug_z = torch.cat((z_i.view(bs, n_dim), adj_z, torch.zeros(bs, n_params).to(z), adj_t[i_t]), dim=-1)

                # Solve augmented system backwards
                aug_ans = ode_solve(aug_z, t_i, t[i_t-1], augmented_dynamics)

                # Unpack solved backwards augmented system
                adj_z[:] = aug_ans[:, n_dim:2*n_dim]
                adj_p[:] += aug_ans[:, 2*n_dim:2*n_dim + n_params]
                adj_t[i_t-1] = aug_ans[:, 2*n_dim + n_params:]

                del aug_z, aug_ans

            ## Adjust 0 time adjoint with direct gradients
            # Compute direct gradients 
            dLdz_0 = dLdz[0]
            dLdt_0 = torch.bmm(torch.transpose(dLdz_0.unsqueeze(-1), 1, 2), f_i.unsqueeze(-1))[:, 0]

            # Adjust adjoints
            adj_z += dLdz_0
            adj_t[0] = adj_t[0] - dLdt_0
        return adj_z.view(bs, *z_shape), adj_t, adj_p, None

In [261]:
class NeuralODE(nn.Module):
    def __init__(self, func):
        super(NeuralODE, self).__init__()
        assert isinstance(func, ODEF)
        self.func = func

    def forward(self, z0, t=Tensor([0., 1.]), return_whole_sequence=False):
        t = t.to(z0)
        # print("the data", t, t.size(), z0.size())
        z = ODEAdjoint.apply(z0, t, self.func.flatten_parameters(), self.func)
        if return_whole_sequence:
            return z
        else:
            return z[-1]

In [262]:
class NNODEF(ODEF):
    def __init__(self, in_dim, hid_dim, time_invariant=False):
        super(NNODEF, self).__init__()
        self.time_invariant = time_invariant
        
        if time_invariant:
            self.lin1 = nn.Linear(in_dim, hid_dim)
        else:
            self.lin1 = nn.Linear(in_dim+1, hid_dim)
        self.lin2     = nn.Linear(hid_dim, hid_dim)
        self.lin3     = nn.Linear(hid_dim, in_dim)
        self.elu      = nn.ELU(inplace=True)

    def forward(self, x, t):
        # print(x.shape, t.shape)
        if not self.time_invariant:
            x = torch.cat((x, t.reshape([1,-1]) ), dim=-1)
        # print(x.shape)
        h = self.elu(self.lin1(x))
        h = self.elu(self.lin2(h))
        out = self.lin3(h)
        return out
    
def to_np(x):
    return x.detach().cpu().numpy()


In [263]:
def conduct_experiment(X, ode_trained, n_steps):
    

    z0 = Variable(torch.Tensor([[0.6, 0.3]]))



    E= X[()]['data'][:,1:]
    h_omega = X[()]['data'][:,0]/50
    N_Max = X[()]['Nmax'].reshape([-1])/18
    n_points =h_omega.shape[0]

 
    def create_batch():
        idx = np.random.randint(0, n_points)      
        ho_ = h_omega[idx].astype(np.float32)
        obs_ = E[idx, :].astype(np.float32)
        ts_ = np.vstack(N_Max).astype(np.float32)
        print(idx, ho_.dtype, obs_.dtype, ts_.dtype)
        return torch.from_numpy(np.array(obs_)), torch.from_numpy(np.array(ts_)), ho_

    # Train Neural ODE
    optimizer = torch.optim.Adam(ode_trained.parameters(), lr=0.01)
    for i in range(n_steps):
        obs_, ts_, ho_ = create_batch()
        # print(obs_[0], torch.tensor(ho_))
        input_d= torch.cat([obs_[0].reshape([1,-1]), torch.tensor(ho_).reshape([1, -1 ])], axis = 1)
        # print(input_d, input_d.shape, ts_)
        z_ = ode_trained(input_d, ts_, return_whole_sequence=True).squeeze(1)
        input_d = torch.cat([obs_.reshape([-1,1]), torch.tensor(np.tile(ho_, 9)).reshape([-1,1])], axis=1)
        # print(z_.shape, input_d.shape)
        loss = F.mse_loss(z_, input_d.detach())
        print("The loss is:", loss.item(), "at step", i)
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

In [264]:
func = NNODEF(2, 16, time_invariant=False)
ode_trained = NeuralODE(func)
conduct_experiment(X, ode_trained, 3000)


13 float32 float32 float32
The loss is: 200.4632568359375 at step 0
17 float32 float32 float32
The loss is: 703.2500610351562 at step 1
1 float32 float32 float32
The loss is: 54.1910285949707 at step 2
10 float32 float32 float32
The loss is: 55.01251983642578 at step 3
11 float32 float32 float32
The loss is: 83.89783477783203 at step 4
11 float32 float32 float32
The loss is: 81.16926574707031 at step 5
13 float32 float32 float32
The loss is: 188.9925537109375 at step 6
10 float32 float32 float32
The loss is: 44.31179428100586 at step 7
9 float32 float32 float32
The loss is: 22.426122665405273 at step 8
17 float32 float32 float32
The loss is: 666.849853515625 at step 9
13 float32 float32 float32
The loss is: 177.4063720703125 at step 10
11 float32 float32 float32
The loss is: 60.83683395385742 at step 11
10 float32 float32 float32
The loss is: 28.76845932006836 at step 12
17 float32 float32 float32
The loss is: 647.0274047851562 at step 13
18 float32 float32 float32
The loss is: 809.751